Surprise는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로도 된 데이터 세트만 적용할 수 있다(무비렌즈 형식)

이와 다를 경우 Reader 클래스를 먼저 설정해야 한다

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\82109/.surprise_data/ml-100k


분리 문자가 탭 문자

In [4]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

- uid : 개별 사용자 아이디
- iid : 영화(또는 아이템) 아이디
- r_ui : 실제 평점
- est : 실제 평점 정보에 기반해 Surprise의 추천 예측 평점
- details : 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용 데이터를 남기는데 사용
    - 'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터
    - 여기서는 모두 False

In [7]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [8]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

#### OS 파일 데이터를 Surprise 데이터 세트로 로딩
- 로딩되는 데이터 파일에 칼럼명을 가지는 header 문자열이 있어서는 안된다.

In [11]:
import pandas as pd

ratings=pd.read_csv('./ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('./ratings_noh.csv', index=False, header=False)

In [12]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('./ratings_noh.csv', reader=reader)

- line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
- sep(char) : 칼럼을 분리하는 분리자, 디폴트는 \t
- rating_scale(tuple, optimal): 평점 값의 최소~최대 평점을 설정. 디폴트는 (1,5)

In [14]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### pandas DataFrame에서 Surprise 데이터 세트로 로딩
- 칼럼 순서 지켜야 함

In [16]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Baseline Rating
- 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것
- 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수

In [18]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ratings.csv')
reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8814  0.8640  0.8684  0.8753  0.8806  0.8739  0.0068  
MAE (testset)     0.6765  0.6630  0.6679  0.6743  0.6758  0.6715  0.0052  
Fit time          5.08    4.91    4.81    5.19    4.98    4.99    0.13    
Test time         0.28    0.14    0.13    0.30    0.13    0.20    0.08    


{'test_rmse': array([0.88144626, 0.86396757, 0.86837759, 0.87525709, 0.88057552]),
 'test_mae': array([0.67645574, 0.663047  , 0.66791885, 0.67434453, 0.67583058]),
 'fit_time': (5.0798633098602295,
  4.909322500228882,
  4.813174724578857,
  5.186029672622681,
  4.976041793823242),
 'test_time': (0.28203725814819336,
  0.14298486709594727,
  0.1327955722808838,
  0.2983512878417969,
  0.12830495834350586)}

In [19]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8747865665378732
{'n_epochs': 20, 'n_factors': 50}


#### 개인화 영화 추천 시스템 구축

In [20]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

- train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생
- 데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스를 이용

In [21]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='./ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [22]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [23]:
movies = pd.read_csv('./movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [24]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [26]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
         '전체 영화 수:', len(total_movies))
    
    return unseen_movies
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [29]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,]
    
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id, title, rating) for id, title, rating in
                      zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
